In [7]:
%pip install graphviz

  Using cached graphviz-0.21-py3-none-any.whl.metadata (12 kB)
Using cached graphviz-0.21-py3-none-any.whl (47 kB)
Note: you may need to restart the kernel to use updated packages.


In [17]:
from graphviz import Digraph

def create_patient_enroll_diagram():
    # 輸出 tiff，等等再用 graph_attr 設 dpi=600
    dot = Digraph(comment='Patient Enrollment', format='tiff')

    # 全域設定：由上往下、直角線、多一點間距
    dot.attr(rankdir='TB', splines='ortho', nodesep='0.8', ranksep='0.8')
    # 圖層屬性：600 dpi + newrank 才吃得到 cross-cluster 的 rank=same
    dot.attr('graph', dpi='600', newrank='true')

    # 節點預設樣式
    dot.attr(
        'node',
        shape='box',
        style='rounded,filled',
        fillcolor='#E3F2FD',
        fontname='Helvetica',
        fontsize='10'
    )

    # ============================
    # Derivation cohort (internal)
    # ============================
    with dot.subgraph(name='cluster_int') as c:
        c.attr(
            label='Derivation cohort (internal hospital)',
            style='dashed',
            color='#1565C0',
            fontcolor='#1565C0'
        )

        # 起始：癌登
        c.node(
            'Reg',
            'Stage III colorectal adenocarcinoma\n'
            'Cancer registry 2017–2021\n'
            'n = 531',
            shape='cylinder',
            fillcolor='#FFECB3'
        )

        # 排除條件
        exclusion_text = (
            'Exclusions (n = 200)\n'
            '• Previous history of colorectal cancer (n = 1)\n'
            '• Double primary cancers (n = 5)\n'
            '• Follow-up < 18 months without distant recurrence (n = 9)\n'
            '• Neoadjuvant chemotherapy (n = 11)\n'
            '• Stage IV disease at diagnosis (n = 1)\n'
            '• Synchronous colorectal cancers (n = 6)\n'
            '• Treated outside institution (no operative/pathology data) (n = 16)\n'
            '• No surgical resection (n = 27)\n'
            '• Rectal cancers (n = 111)\n'
            '• Non-R0 resection (n = 7)\n'
            '• Surgery performed in 2022 (n = 6)'
        )
        c.node('Excl', exclusion_text, fillcolor='#E3F2FD')

        # 最終 internal cohort
        c.node(
            'FinalInt',
            'Final analytic cohort\n'
            'Stage III colon adenocarcinoma\n'
            'Curative-intent colectomy\n'
            'n = 331',
            fillcolor='#C8E6C9'
        )

        c.edge('Reg', 'Excl')
        c.edge('Excl', 'FinalInt')

    # ============================
    # External validation cohort
    # ============================
    with dot.subgraph(name='cluster_ext') as c:
        c.attr(
            label='External validation cohort',
            style='dashed',
            color='#2E7D32',
            fontcolor='#2E7D32'
        )

        c.node(
            'ExtStart',
            'Independent tertiary centre\n'
            'Stage III colon adenocarcinoma\n'
            'n = TBD',
            shape='cylinder',
            fillcolor='#FFECB3'
        )

        c.node(
            'ExtFinal',
            'External analytic cohort\n'
            'After applying same inclusion/exclusion\n'
            'n = TBD',
            fillcolor='#C8E6C9'
        )

        c.edge('ExtStart', 'ExtFinal')

    # 讓兩個 final cohort 在同一橫列（左右對齊感會好很多）
    dot.body.append('{ rank=same; FinalInt ExtFinal }')

    return dot

# 產生並輸出 600 dpi TIFF（同時留一份 PDF/PNG 方便預覽也可以）
patient_diag = create_patient_enroll_diagram()
patient_diag.render('patient_enrollment_flowchart', view=True, cleanup=True)
print("Patient enrollment flowchart 已產生：patient_enrollment_flowchart.tiff")

Patient enrollment flowchart 已產生：patient_enrollment_flowchart.tiff
